In [ ]:
import pandas as pd
from openai import OpenAI
import numpy as np

In [ ]:
client = OpenAI(api_key)

In [ ]:
#df = pd.read_csv('dataset2.csv')
#df.head()

,text
0,"Abdominal Hernia: Visible bulge or swelling, d..."
1,"Abdominal Injury: Abdominal pain, tenderness, ..."
2,"Abscess: Swelling, pain, redness, fever, pus f..."
3,"Abscess of Nose: Pain, swelling, redness, disc..."
4,"Abscess of the Lung: Cough with phlegm or pus,..."


In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
  text = text.replace("\n", "")
  return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
#get_embedding(df['text'].iloc[0]) # embedding of the first line of text column

[-0.032867081463336945,
 0.044648800045251846,
 0.030997658148407936,
 0.09355814754962921,
 -0.0021370667964220047,
 0.009439503774046898,
 0.0019387122010812163,
 -0.004227941390126944,
 0.02645452320575714,
 0.015433616004884243,
 0.048822395503520966,
 0.019378969445824623,
 -0.028345683589577675,
 -0.01681394688785076,
 0.014140235260128975,
 0.08921065181493759,
 0.02171575091779232,
 0.013998942449688911,
 -0.04388798773288727,
 0.03254101797938347,
 0.06521246582269669,
 0.01890074461698532,
 0.024802472442388535,
 -0.012846855446696281,
 0.03827971592545509,
 0.022161368280649185,
 -0.012303417548537254,
 -0.004445316269993782,
 0.041866399347782135,
 0.015151028521358967,
 0.05838689208030701,
 -0.015085816383361816,
 0.03697546571493149,
 -0.026845797896385193,
 -0.030215106904506683,
 0.04534439742565155,
 -0.00259083672426641,
 0.03558426722884178,
 0.027867458760738373,
 0.009031926281750202,
 -0.0070538148283958435,
 0.02567197196185589,
 0.012488186359405518,
 0.0258893

In [ ]:
df['embeddings'] = df['text'].apply(get_embedding)
df.to_csv('dataset2_embeddings.csv', index=False)
df.to_pickle('dataset2_embeddings.pkl')

In [ ]:
df_input = pd.read_csv('sampled_df.csv')
df_embed = pd.read_pickle('dataset2_embeddings.pkl')

In [ ]:
def diagnose(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful doctor, response to the input symptoms and ask questions to get more details if needed." },
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages
    )
    return response.choices[0].message.content

In [ ]:
diagnoses = []

format = "\nGive me a list of 4 possible illnesses. The output should in format: '\nIllness List:\n    1. Disease_1\n    2. Disease_2\n    3. Disease_3\n    4. Disease_4'"

In [ ]:
for i in range(0, 2):
    history_transcript = ''
    print(f'Patient {i+1}')
    input_history = ''
    n = 1
    user_input = df_input.iloc[i]['sentence_'+str(n)]
    while user_input != 'no more information':
        input_history += user_input + '\n'
        history_transcript += f'Patient: {user_input}\n'
        print(f'Patient: {user_input}\n')
        response = diagnose(user_input)

        print(f'Doctor: {response}\n')
        history_transcript += f'Doctor: {response}\n'

        n += 1
        user_input = df_input.iloc[i]['sentence_'+str(n)]
    input_embedding = get_embedding(input_history)

    df_embed['similarity'] = df_embed['embeddings'].apply(lambda x: np.dot(x, input_embedding))
    df_embed.sort_values('similarity', ascending=False, inplace=True)

    # top 4 relavent text
    context = df_embed['text'].iloc[0] + '\n' + df_embed['text'].iloc[1] + '\n' + df_embed['text'].iloc[2] + '\n' + df_embed['text'].iloc[3]
    rag_prompt = f"Here are some information that maybe helpful for the diagnosis:\n{context}"

    print(f'Patient: {user_input}\n')
    print(rag_prompt + '\n')
    response = diagnose(history_transcript + rag_prompt + format)
    print(f'Doctor: {response}\n')
    diagnoses.append(response)

    print('\n------------------------------\n')

Patient 1
Patient: My skin is itchy and red.
Doctor: I'm sorry to hear that you're experiencing this discomfort. Itchy, red skin can be caused by a variety of factors. Could you provide some more information to help me understand your situation better?

1. When did you first notice these symptoms?
2. Are there any specific areas on your body that are affected?
3. Have you recently used any new skincare products, detergents, or have you been in contact with any potential irritants or allergens?
4. Do you have any other symptoms, such as rash, swelling, or blistering?
5. Have you had any similar symptoms in the past?
Patient: Sometimes, it can peel.
Doctor: I understand you're experiencing some peeling, which can be concerning. Could you provide more details to help me understand better?

1. Where exactly is the peeling occurring on your body?
2. Have you noticed any other symptoms in the affected area, such as redness, itching, or pain?
3. When did you first notice the peeling, and has 

In [ ]:
print(diagnoses)

['Illness List:\n1. Allergic Reaction\n2. Eczema\n3. Contact Dermatitis\n4. Sinusitis with Allergic Component', "Based on the symptoms you've described, here are some possible illnesses that could be contributing to your condition:\n\nIllness List:\n1. Allergic Rhinitis\n2. Acute Sinusitis\n3. Chronic Sinusitis\n4. Angioedema\n\nThese possibilities consider the combination of symptoms like sneezing, facial swelling, and nasal issues. However, a thorough evaluation in person would provide a more accurate diagnosis."]


In [ ]:
input_history = ''
input_embedding = get_embedding(input_history)

df_embed['similarity'] = df_embed['embeddings'].apply(lambda x: np.dot(x, input_embedding))
df_embed.sort_values('similarity', ascending=False, inplace=True)

# top 4 relavent text
context = df_embed['text'].iloc[0] + '\n' + df_embed['text'].iloc[1] + '\n' + df_embed['text'].iloc[2] + '\n' + df_embed['text'].iloc[3]